In [ ]:
#| default_exp transformers.mha

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

#| hide
#| export

from fastcore.basics import patch
from nbdev.showdoc import *

# Multi-Headed Attention (MHA) Transformer

> Annotated [PyTorch](https://pytorch.org) implementation of the standard Multi-Headed Attention (MHA) Transformer architecture, introduced in the paper ["Attention is All You Need"](https://arxiv.org/abs/1706.03762).

The implementation is inspired by [Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html).

In [ ]:
#| hide 
#| export

import math
from typing import Optional, List

import torch
import torch.nn as nn

from labml import tracker

In [ ]:
#| exports

class PrepareForMHA(nn.Module):
    "Prepare for multi-head attention."

    def __init__(self, 
                 d_model: int, # Number of features in the `query`, `key` and `value` vectors.
                 heads: int, # Number of heads
                 d_k: int, # Number of dimensions in vectors on each head
                 bias: bool
                ):
        
        super().__init__()
        # Linear layer for linear transform
        self.linear = nn.Linear(d_model, heads * d_k, bias=bias)
        
        self.heads = heads
        self.d_k = d_k

The `PrepareForHMA` module performs a linear transformation and splits the input vector into given number of heads for multi-head attention. This is used to transform **key**, **query**, and **value** vectors.

In [ ]:
#| exports

@patch
def forward(self:PrepareForMHA, 
            x: torch.Tensor # Input has shape `[seq_len, batch_size, d_model]` or `[batch_size, d_model]`
            ) -> torch.Tensor: # Output has shape `[seq_len, batch_size, heads, d_k]` or `[batch_size, heads, d_model]`
    "Forward pass."
    
    # We apply the linear transformation to the last dimension and split that into the heads.
    head_shape = x.shape[:-1]
    # Linear transform
    x = self.linear(x)
    # Split last dimension into heads
    x = x.view(*head_shape, self.heads, self.d_k)

    return x

The `forward` pass of `PrepareForHMA` applies the linear transformation to the last dimension and split that into the heads.

In [ ]:
#| exports

class MultiHeadAttention(nn.Module):
    "A Multi-Head Attention Module."

    def __init__(self, 
                 heads: int, # Number of heads
                 d_model: int, # Number of features in the `query`, `key` and `value` vectors.
                 dropout_prob: float = 0.1, 
                 bias: bool = True # Whether to have a bias parameter for transformations for Q, K and V
                ):

        super().__init__()

        # Number of features per head
        self.d_k = d_model // heads
        # Number of heads
        self.heads = heads

        # These transform the `query`, `key` and `value` vectors for multi-headed attention.
        self.query = PrepareForMHA(d_model, heads, self.d_k, bias=bias)
        self.key = PrepareForMHA(d_model, heads, self.d_k, bias=bias)
        self.value = PrepareForMHA(d_model, heads, self.d_k, bias=True)

        # Softmax for attention along the time dimension of `key`
        self.softmax = nn.Softmax(dim=1)

        # Output layer
        self.output = nn.Linear(d_model, d_model)
        # Dropout
        self.dropout = nn.Dropout(dropout_prob)
        # Scaling factor before the softmax
        self.scale = 1 / math.sqrt(self.d_k)

        # We store attentions so that it can be used for logging, or other computations if needed
        self.attn = None


In [ ]:
#| export

@patch
def get_scores(self:MultiHeadAttention, 
               query: torch.Tensor, # Input query
               key: torch.Tensor # Input key
               ) -> torch.Tensor:
        
        "Calculate scores between queries and keys."

        return torch.einsum('ibhd,jbhd->ijbh', query, key)

`MultiHeadAttention.get_scores()` computes $Q K^\top$ or $S_{ijbh} = \sum_d Q_{ibhd} K_{jbhd}$. This method can be overridden for other variations like relative attention.

In [ ]:
#| export

@patch
def prepare_mask(self:MultiHeadAttention, 
                 mask: torch.Tensor, # It has shape `[seq_len_q, seq_len_k, batch_size]`, where first dimension is the query dimension.
                 query_shape: List[int], 
                 key_shape: List[int]
                ) -> torch.Tensor: # Resulting mask has shape `[seq_len_q, seq_len_k, batch_size, heads]`
    
    "Prepare mask for MHA."
    # If the query dimension is equal to $1$ it will be broadcasted.

    assert mask.shape[0] == 1 or mask.shape[0] == query_shape[0]
    assert mask.shape[1] == key_shape[0]
    assert mask.shape[2] == 1 or mask.shape[2] == query_shape[1]

    # Same mask applied to all heads.
    mask = mask.unsqueeze(-1)

    return mask

In [ ]:
#| exports

@patch
def forward(self:MultiHeadAttention,
            query: torch.Tensor, # Tensor that stores collection of *query* vectors
            key: torch.Tensor, # Tensor that stores collection of *key* vectors
            value: torch.Tensor, # Tensor that stores collection of *value* vectors
            mask: Optional[torch.Tensor] = None
            ):
    "Forward pass."

    # `query`, `key` and `value`  have shape `[seq_len, batch_size, d_model]`
    seq_len, batch_size, _ = query.shape

    if mask is not None:
        mask = self.prepare_mask(mask, query.shape, key.shape)

    # Prepare `query`, `key` and `value` for attention computation.
    # These will then have shape `[seq_len, batch_size, heads, d_k]`.
    query = self.query(query)
    key = self.key(key)
    value = self.value(value)

    scores = self.get_scores(query, key) # <1>
    scores *= self.scale # <2>
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf')) # <3>

    attn = self.softmax(scores) # <4>

    # Save attentions if debugging
    tracker.debug('attn', attn)

    # Apply dropout
    attn = self.dropout(attn)

    x = torch.einsum("ijbh,jbhd->ibhd", attn, value) # <5>

    # Save attentions for any other calculations 
    self.attn = attn.detach()

    # Concatenate multiple heads
    x = x.reshape(seq_len, batch_size, -1)

    # Output layer
    return self.output(x)

1. Compute attention scores $Q K^\top$. This gives a tensor of shape `[seq_len, seq_len, batch_size, heads]`
2. Scale scores $\frac{Q K^\top}{\sqrt{d_k}}$
3. Apply mask
4. $softmax$ attention along the key sequence dimension $\underset{seq}{softmax}\Bigg(\frac{Q K^\top}{\sqrt{d_k}}\Bigg)$
5. Multiply by values $\underset{seq}{softmax}\Bigg(\frac{Q K^\top}{\sqrt{d_k}}\Bigg)V$

The `forward` pass of `MultiHeadAttention` computes scaled multi-headed attention for given `query`, `key` and `value` vectors. `query`, `key` and `value` are the tensors that store collection of *query*, *key* and *value* vectors. They have shape `[seq_len, batch_size, d_model]`, `mask` has shape `[seq_len, seq_len, batch_size]` and `mask[i, j, b]` indicates whether for batch `b`, query at position `i` has access to key-value at position `j`.

$$\mathop{Attention}(Q, K, V) = \underset{seq}{\mathop{softmax}}\Bigg(\frac{Q K^\top}{\sqrt{d_k}}\Bigg)V$$

In simple terms, it finds keys that matches the query, and gets the values of those keys. It uses dot-product of query and key as the indicator of how matching they are. Before taking the $softmax$ the dot-products are scaled by $\frac{1}{\sqrt{d_k}}$. This is done to avoid large dot-product values causing softmax to give very small gradients when $d_k$ is large. Softmax is calculated along the axis of of the sequence (or time).

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()